In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=a3d864a9b8458f5f8e6d5fbdc22478ad45a6d9aa44d2a3f57cb5b98a6314219f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark import SparkConf,SparkContext

In [3]:
conf=SparkConf().setAppName("Assignment 3").setMaster("local[*]")
sc=SparkContext(conf=conf)

##Q1


In [60]:
def number_of_consecutive(line):
  d={}
  count=0
  comb={}
  for i in range(len(line)-2):
    if len(line[i]) >= 4 and len(line[i+1]) >= 4 and len(line[i+2]) >= 4:
      if line[i][0].lower() == line[i+1][0].lower() == line[i+2][0].lower():
         # Part a)
        count += 1  

        # Part b)
        letter=line[i][0]  
        if letter.upper() in d:
          d[letter.upper()] += 1
        else:
          d[letter.upper()]=1  
         
        # Part c) 
        key=(line[i],line[i+1],line[i+2])   
        if key in comb:
          comb[key] +=1
        else:
          comb[key] =1
 
  return count,d,comb   

file_rdd = sc.textFile("q1.txt").flatMap(lambda x: x.split())
file_rdd = file_rdd.map(lambda word: [word]).reduce(lambda a, b: a + b)
count,dic,combination = number_of_consecutive(file_rdd)
print(f"Total Count = {count}")
for i,j in dic.items():
   print(f"{i} => {j}")
for i,j in combination.items():
  print(f"{i} => {j}")


Total Count = 8
H => 3
M => 1
S => 1
D => 2
F => 1
('Horrid', 'Henry’s', 'hound') => 2
('Henry’s', 'hound', 'hunts') => 1
('massive', 'Murree', 'mountains.') => 1
('silly', 'stupid', 'Samuel’s') => 1
('dark', 'dreadful', 'dragon') => 1
('dreadful', 'dragon', 'Dany') => 1
('fantastic', 'fanciful', 'foursome.') => 1


###Q2

In [61]:
def split(l):
  return l[0],l[2:]

def split2(l):
  return l[0:2],l[3:]

def Simple_matching_cofficient(list1,list2):
    m11=0
    m00=0
    m01=0
    m10=0

    for (i,j) in zip(list1,list2):
      if i==1 and j==1:
        m11 += 1
      elif i==0 and j==0:
        m00 += 1
      elif i==0 and j==1:
        m01 += 1
      elif i==1 and j==0:
        m10 += 1
    SMC=(m11+m00)/(m11+m00+m10+m01)
    return SMC

def fun1(l):
    li=[]
    for i in l:
      if i==" ":
        continue
      else:
        li.append(int(i))  
    return li 


   
def question2(line,words):

    a=[]
    b=[]
    c=[]
    list1=[]
    list2=[]

    for i in line:

      a=split(i)
      list1=fun1(a[1])

      for j in words:
        b=split2(j)
        list2=fun1(b[1])
        d=Simple_matching_cofficient(list1,list2)
        c.append(d)

      if c[0] >= c[1]:
        print(a[0],"=> R1" )  
      else:
        print(a[0],"=> R2" )

      c.clear()   

input=sc.textFile("input.txt")
reference=sc.textFile("refrence.txt")
input= input.map(lambda word: [word]).reduce(lambda a, b: a + b)
reference= reference.map(lambda word: [word]).reduce(lambda a, b: a + b)
output=question2(input,reference)      
    

1 => R1
2 => R1
3 => R1
4 => R2


##Q3


In [67]:
author = sc.textFile("author.txt")
co_author = author.map(lambda line: line.split('->'))
common_coauthor = co_author.map(lambda x: (x[1], x[0])).distinct().groupByKey().mapValues(lambda x: len(x))

for author, count in common_coauthor.collect():
    row = common_coauthor.filter(lambda x: x[0] == author).first()
    print(f"{author} ({count}): {row[1]}")


 B. Cao, C. Rego, F. Glover, K. Kiani (1): 1
 B. Cao, Y. Lu (2): 2
 K. Kiani (1): 1
 C. Rego, F. Glover, K. Kiani, Y. Lu (1): 1
 B. Hosseini, B. Cao, Y. Lu (1): 1
